In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/home-credit-credit-risk-model-stability/sample_submission.csv
/kaggle/input/home-credit-credit-risk-model-stability/feature_definitions.csv
/kaggle/input/home-credit-credit-risk-model-stability/parquet_files/test/test_deposit_1.parquet
/kaggle/input/home-credit-credit-risk-model-stability/parquet_files/test/test_applprev_2.parquet
/kaggle/input/home-credit-credit-risk-model-stability/parquet_files/test/test_static_cb_0.parquet
/kaggle/input/home-credit-credit-risk-model-stability/parquet_files/test/test_static_0_0.parquet
/kaggle/input/home-credit-credit-risk-model-stability/parquet_files/test/test_credit_bureau_a_1_3.parquet
/kaggle/input/home-credit-credit-risk-model-stability/parquet_files/test/test_credit_bureau_a_1_2.parquet
/kaggle/input/home-credit-credit-risk-model-stability/parquet_files/test/test_tax_registry_b_1.parquet
/kaggle/input/home-credit-credit-risk-model-stability/parquet_files/test/test_static_0_2.parquet
/kaggle/input/home-credit-credit-risk-model-st

In [2]:
import pandas as pd
import numpy as np
import os
import sys
import psutil
from datetime import datetime

In [3]:
# !pip install polars==0.20.26
import polars as pl
pl.show_versions()

--------Version info---------
Polars:               0.20.21
Index type:           UInt32
Platform:             Linux-5.15.133+-x86_64-with-glibc2.31
Python:               3.10.13 | packaged by conda-forge | (main, Dec 23 2023, 15:36:39) [GCC 12.3.0]

----Optional dependencies----
adbc_driver_manager:  <not installed>
cloudpickle:          2.2.1
connectorx:           <not installed>
deltalake:            <not installed>
fastexcel:            <not installed>
fsspec:               2024.2.0
gevent:               <not installed>
hvplot:               <not installed>
matplotlib:           3.7.5
nest_asyncio:         1.5.8
numpy:                1.26.4
openpyxl:             3.1.2
pandas:               2.2.2
pyarrow:              15.0.2
pydantic:             2.5.3
pyiceberg:            <not installed>
pyxlsb:               <not installed>
sqlalchemy:           2.0.25
xlsx2csv:             <not installed>
xlsxwriter:           <not installed>


In [4]:
!mkdir /kaggle/working/intermediate

!mkdir /kaggle/working/intermediate/train/
!mkdir /kaggle/working/intermediate/test/

!mkdir /kaggle/working/intermediate/train/base
!mkdir /kaggle/working/intermediate/test/base

!mkdir /kaggle/working/intermediate/train/applprev_1
!mkdir /kaggle/working/intermediate/test/applprev_1

!mkdir /kaggle/working/intermediate/train/other_1
!mkdir /kaggle/working/intermediate/test/other_1

!mkdir /kaggle/working/intermediate/train/tax_registry
!mkdir /kaggle/working/intermediate/test/tax_registry

In [5]:
SAVE_PATH = "/kaggle/working/intermediate"
DATA_PATH = "/kaggle/input/home-credit-credit-risk-model-stability/parquet_files"

In [6]:
def return_test_submission():
    test_base = pd.read_parquet("/kaggle/input/home-credit-credit-risk-model-stability/parquet_files/test/test_base.parquet")
    pred = []
    for i in range(test_base.shape[0]):
        pred.append(abs(np.random.randn()))
    score = pd.DataFrame({"score": pred})
    submission = pd.concat([test_base[["case_id",]], score], axis=1)
    submission.to_csv('submission.csv', index=False)

In [7]:
def clean_columns(df, threshold, columns): #threshold (0, 1) percentage
    arr = [col for col in df.columns if df.select( pl.col(col).is_null().sum()/df[col].shape[0] < threshold ).row(0)[0]]
    return arr

def fill_na(df):
    return (df
            .fill_null(strategy="forward")
            .fill_null(strategy="backward")
            .select(pl.col("case_id"),
                    pl.col(pl.Float64, pl.Int64).exclude("case_id").fill_nan('mean').cast(pl.Float64),
                    pl.col(pl.String),
                    pl.col(pl.Boolean))
           )

In [8]:
train = pl.read_parquet(os.path.join(DATA_PATH, 'train/train_base.parquet'))
test = pl.read_parquet(os.path.join(DATA_PATH, 'test/test_base.parquet'))

train.write_parquet(os.path.join(SAVE_PATH, 'train/base/train_base.parquet'))
test.write_parquet(os.path.join(SAVE_PATH, 'test/base/test_base.parquet'))

In [9]:
avg = ['actualdpd_943P','annuity_853A', 'byoccupationinc_3656910L', 'childnum_21L', 'credacc_actualbalance_314A',
       'credacc_credlmt_575A','credacc_maxhisbal_375A','credacc_minhisbal_90A', 'credacc_transactions_402L', 'credamount_590A',
       'currdebt_94A', 'mainoccupationinc_437A', 'pmtnum_8L','tenor_203L']

mode = ['credtype_587L', 'district_544M', 'downpmt_134A', 'education_1138M', 'familystate_726L',
        'inittransactioncode_279L', 'rejectreason_755M', 'status_219L']

train_0 = pl.read_parquet(os.path.join(DATA_PATH, 'train/train_applprev_1_0.parquet'))
select_columns = clean_columns(train_0, 0.1, train_0.columns)
train_0 = train_0.select( pl.col(select_columns))
train_0 = fill_na(train_0)

train_0_ready = (train_0
 .group_by("case_id")
 .agg(pl.col([col for col in avg if (col in select_columns)]).mean(),
      pl.col([col for col in mode if col in select_columns]).mode().first())
 .sort("case_id")
 )

train_1 = pl.read_parquet(os.path.join(DATA_PATH, 'train/train_applprev_1_1.parquet'))
train_1 = train_1.select( pl.col(select_columns))
train_1 = fill_na(train_1)

train_1_ready = (train_1
 .group_by("case_id")
 .agg(pl.col([col for col in avg if (col in select_columns)]).mean(),
      pl.col([col for col in mode if col in select_columns]).mode().first())
 .sort("case_id")
 )

train = pl.concat([train_0_ready, train_1_ready]).group_by("case_id").agg(pl.all().mode().first()) 
train.write_parquet(os.path.join(SAVE_PATH, "train/applprev_1/train_applprev_1.parquet"))

In [10]:
#test_applprev_1_0
test_0 = pl.read_parquet(os.path.join(DATA_PATH, 'test/test_applprev_1_0.parquet'))
test_0 = test_0.select( pl.col(select_columns))
test_0 = fill_na(test_0)

test_0_ready = (test_0
 .group_by("case_id")
 .agg(pl.col([col for col in avg if (col in select_columns)]).mean(),
      pl.col([col for col in mode if col in select_columns]).mode().first())
 .sort("case_id")
#  .write_parquet(os.path.join(SAVE_PATH, 'test/applprev_1/test_applprev_1_0.parquet'))
)

#test_applprev_1_1
test_1 = pl.read_parquet(os.path.join(DATA_PATH, 'test/test_applprev_1_1.parquet'))
test_1 = test_1.select( pl.col(select_columns))
test_1 = fill_na(test_1)

test_1_ready = (test_1
 .group_by("case_id")
 .agg(pl.col([col for col in avg if (col in select_columns)]).mean(),
      pl.col([col for col in mode if col in select_columns]).mode().first())
 .sort("case_id")
#  .write_parquet(os.path.join(SAVE_PATH, 'test/applprev_1/test_applprev_1_1.parquet'))
 )

#test_applprev_1_2
test_2 = pl.read_parquet(os.path.join(DATA_PATH,'test/test_applprev_1_2.parquet'))
test_2 = test_2.select( pl.col(select_columns))
test_2 = fill_na(test_2)
''
test_2_ready = (test_2
 .group_by("case_id")
 .agg(pl.col([col for col in avg if (col in select_columns)]).mean(),
      pl.col([col for col in mode if col in select_columns]).mode().first())
 .sort("case_id")
#  .write_parquet(os.path.join(SAVE_PATH, 'test/applprev_1/test_applprev_1_2.parquet'))
 )

test = pl.concat([test_0_ready, test_1_ready, test_2_ready]).group_by("case_id").agg(pl.all().mode().first()) 
train.write_parquet(os.path.join(SAVE_PATH, "test/applprev_1/test_applprev_1.parquet"))

In [11]:
#train_tax_registry
train_a = pl.read_parquet(os.path.join(DATA_PATH, 'train/train_tax_registry_a_1.parquet'))
train_b = pl.read_parquet(os.path.join(DATA_PATH, 'train/train_tax_registry_b_1.parquet'))
train_c = pl.read_parquet(os.path.join(DATA_PATH, 'train/train_tax_registry_c_1.parquet'))

train_b = train_b.rename({
    "amount_4917619A": "amount_4527230A",
    "name_4917606M": "name_4527232M",
    "deductiondate_4917603D": "recorddate_4527225D"
})
train_c = train_c.rename({
    "pmtamount_36A": "amount_4527230A",
    "employername_160M": "name_4527232M",
    "processingdate_168D": "recorddate_4527225D"
})

select_columns = clean_columns(train_a, 0.1, train_a.columns)

train_a = train_a.select(pl.col(select_columns))
train_b = train_b.select(pl.col(select_columns))
train_c = train_c.select(pl.col(select_columns))

#fill_na
train_a = fill_na(train_a)
train_b = fill_na(train_b)
train_c = fill_na(train_c)

#first file in train registry
train_a = (train_a
 .group_by("case_id")
 .agg(pl.col(pl.Float64, pl.Int64).exclude("case_id", "num_group1").mean(),
      pl.col(pl.String, pl.Boolean).mode().first())
#  .select(pl.all().is_nan().sum())
 .select(pl.all().exclude("recorddate_4527225D")) #this feature is not of much use in absence of some other date feature to compare it with
 .sort("case_id")
 )

#second file in train registry
train_b = (train_b
 .group_by("case_id")
 .agg(pl.col(pl.Float64, pl.Int64).exclude("case_id", "num_group1").mean(),
      pl.col(pl.String, pl.Boolean).mode().first())
#  .select(pl.all().is_nan().sum())
 .select(pl.all().exclude("recorddate_4527225D")) #this feature is not of much use in absence of some other date feature to compare it with
 .sort("case_id")
 )

#third file in train registry
train_c = (train_c
 .group_by("case_id")
 .agg(pl.col(pl.Float64, pl.Int64).exclude("case_id", "num_group1").mean(),
      pl.col(pl.String, pl.Boolean).mode().first())
#  .select(pl.all().is_nan().sum())
 .select(pl.all().exclude("recorddate_4527225D")) #this feature is not of much use in absence of some other date feature to compare it with
 .sort("case_id")
 )

#a, b and c contain some records having same case_id
train = pl.concat([train_a, train_b, train_c]).group_by("case_id").agg(pl.all().mode().first()) 
train.write_parquet(os.path.join(SAVE_PATH, 'train/tax_registry/train_tax_registry.parquet'))

In [12]:
#test tax registry
test_a = pl.read_parquet(os.path.join(DATA_PATH, 'test/test_tax_registry_a_1.parquet'))
test_b = pl.read_parquet(os.path.join(DATA_PATH, 'test/test_tax_registry_b_1.parquet'))
test_c = pl.read_parquet(os.path.join(DATA_PATH, 'test/test_tax_registry_c_1.parquet')) #is empty

test_b = test_b.rename({
    "amount_4917619A": "amount_4527230A",
    "name_4917606M": "name_4527232M",
    "deductiondate_4917603D": "recorddate_4527225D"
})

#fill_na
test_a = fill_na(test_a)
test_b = fill_na(test_b)

#grouping and aggregating
test_a = (test_a
 .group_by("case_id")
 .agg(pl.col(pl.Float64, pl.Int64).exclude("case_id", "num_group1").mean(),
      pl.col(pl.String, pl.Boolean).mode().first())
#  .select(pl.all().is_nan().sum())
 .select(pl.all().exclude("recorddate_4527225D")) #this feature is not of much use in absence of some other date feature to compare it with
 .sort("case_id")
 )

test_b = (test_b
 .group_by("case_id")
 .agg(pl.col(pl.Float64, pl.Int64).exclude("case_id", "num_group1").mean(),
      pl.col(pl.String, pl.Boolean).mode().first())
#  .select(pl.all().is_nan().sum())
 .select(pl.all().exclude("recorddate_4527225D")) #this feature is not of much use in absence of some other date feature to compare it with
 .sort("case_id")
 )

#writing to disk
test = pl.concat([test_a, test_b]).group_by("case_id").agg(pl.all().mode().first()) #a and b contain some records having same case_id
test.write_parquet(os.path.join(SAVE_PATH, 'test/tax_registry/test_tax_registry.parquet'))

In [13]:
base = pl.read_parquet(os.path.join(SAVE_PATH, "train/base/train_base.parquet"))
tax_registry = pl.read_parquet(os.path.join(SAVE_PATH, "train/tax_registry/train_tax_registry.parquet"))
# applprev_1 = pl.read_parquet(os.path.join(SAVE_PATH, "train/applprev_1/train_applprev_1.parquet"))

train = (base
         .join(tax_registry, on="case_id", how="left")
#          .join(applprev_1, on="case_id", how="inner")
         .select(pl.all().exclude("date_decision"))
         )
train = fill_na(train)

train_cat = (train
            .select(pl.col("case_id"), pl.col(pl.String))
            )
train_num = (train
             .select(pl.all().exclude(pl.String, pl.Boolean), pl.col(pl.Boolean).cast(pl.Int64))
            )

In [14]:
#encoding and scaling

from sklearn.preprocessing import OrdinalEncoder
ord_encoder = OrdinalEncoder()

train_cat_encoded = pd.DataFrame(ord_encoder.fit_transform(train_cat.drop("case_id")), columns=train_cat.columns[1:])
train_cat_encoded["case_id"] = train_cat["case_id"]

train_num_pd = train_num.to_pandas()

train_encoded = (train_num_pd
                 .merge(train_cat_encoded, how="inner", on="case_id")
                )

from sklearn.preprocessing import StandardScaler
std_scaler = StandardScaler()
feat_scale = [col for col in train_encoded.columns if col not in ["case_id", "target"]]
train_encoded_scaled = pd.DataFrame(std_scaler.fit_transform(train_encoded[feat_scale]), columns=feat_scale)
train_encoded_scaled[["case_id", "target"]] = train_encoded[["case_id", "target"]]

In [15]:
train_encoded_scaled

,MONTH,WEEK_NUM,amount_4527230A,name_4527232M,case_id,target
0,-0.788806,-1.713131,-0.371996,1.038520,0,0.0
1,-0.788806,-1.713131,-0.371996,1.038520,1,0.0
2,-0.788806,-1.713131,-0.371996,1.038520,2,0.0
3,-0.788806,-1.713131,-0.371996,1.038520,3,0.0
4,-0.788806,-1.713131,-0.371996,1.038520,4,1.0
...,...,...,...,...,...,...
1526654,1.647713,2.110724,-0.200735,-0.458398,2703450,0.0
1526655,1.647713,2.110724,-0.200735,-0.458398,2703451,0.0
1526656,1.647713,2.110724,0.509529,0.649967,2703452,0.0
1526657,1.647713,2.110724,0.509529,0.649967,2703453,0.0


In [16]:
# #train test split

# from sklearn.model_selection import train_test_split
# X_train, X_val, y_train, y_val = train_test_split(train_encoded_scaled.drop(columns=["target"]), \
#                                                   train_encoded_scaled["case_id"], test_size=0.2, random_state=42)

In [17]:

import vaex as vx
import vaex.ml


In [18]:
#model training

from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression(max_iter=100)

In [19]:
train = vx.from_pandas(train_encoded_scaled)
from vaex.ml.sklearn import IncrementalPredictor
from sklearn.linear_model import SGDClassifier
features = [col for col in train.columns if col not in ["case_id", "target"]]
target = 'target'
model = SGDClassifier(loss="modified_huber")
vaex_model = IncrementalPredictor(model=model, features=features, target=target, batch_size=100000,
                                 partial_fit_kwargs={'classes':[0, 1]})


In [20]:
vaex_model.fit(df=train, progress='widget')

# Test

In [21]:

base = pl.read_parquet(os.path.join(SAVE_PATH, "test/base/test_base.parquet"))
tax_registry = pl.read_parquet(os.path.join(SAVE_PATH, "test/tax_registry/test_tax_registry.parquet"))
# applprev_1 = pl.read_parquet(os.path.join(SAVE_PATH, "test/applprev_1/test_applprev_1.parquet"))


test = (base
         .join(tax_registry, on="case_id", how="left")
#          .join(applprev_1, on="case_id", how="inner")
         .select(pl.all().exclude("date_decision"))
         )

test = fill_na(test)

test_cat = (test
                .select(pl.col("case_id"), pl.col(pl.String))
                )

test_num = (test
             .select(pl.all().exclude(pl.String, pl.Boolean), pl.col(pl.Boolean).cast(pl.Int64))
            )

In [25]:
# the error is in one of these two lines

# try:
temp_df = test_cat.drop("case_id").to_pandas()
test_cat_np = ord_encoder.fit_transform(temp_df)
test_cat_encoded = pd.DataFrame(test_cat_np, columns=test_cat.columns[1:])
test_cat_encoded["case_id"] = test_cat["case_id"]
test_num_pd = test_num.to_pandas()

test_encoded = (test_num_pd
                 .merge(test_cat_encoded, how="inner", on="case_id")
                )

feat_scale = [col for col in test_encoded.columns if col not in ["case_id"]]
test_encoded_scaled = pd.DataFrame(std_scaler.transform(test_encoded[feat_scale]), columns=feat_scale)
test_encoded_scaled[["case_id"]] = test_encoded[["case_id"]]

test = vx.from_pandas(test_encoded_scaled)

submission = pd.DataFrame({
    "case_id": test_encoded_scaled["case_id"],
    "score": vaex_model.predict(test)
})

submission.to_csv("submission.csv", index=False)
# except:
#     print("No")

In [26]:
#The error which had been a pain in the ass was caused by the oridinal encoder. 
# It threw the error when it encountered an unseen category in the hidden test set.